<center>
<p><img src="https://www.gob.mx/cms/uploads/image/file/179499/outstanding_quienes-somos.jpg" width="300">
</p>



# Curso *Machine Learning con uso de pandas, scikit learn y libretas jupyter*

# Pequeño taller datos diarios 


<p> Julio Waissman Vilanova </p>
<p>
<img src="https://identidadbuho.unison.mx/wp-content/uploads/2019/06/letragrama-cmyk-72.jpg" width="80">
</p>
</center>


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (15,7)

# Adquiriendo los datos

In [ ]:
url = "https://github.com/juliowaissman/curso-ml-cenace/raw/main/datos/Dataset_GCRNO_05052021.xlsx"

df = pd.read_excel(url, sheet_name='Datos')
df_dic = pd.read_excel(url, sheet_name='Descripción de Variables')

df.info()

In [ ]:
df

In [ ]:
df_dic

Como vemos, se tiene la demanda por 24 horas, pero se encuentra en una misma columna. Si quisieramos utilizarlo para hacer el forecasting de la demanda integrada en forma horaria, necesitariamos cambiar esas columnas a instancias, lo que se puede hacer en pandas mas o menos fácil.

In [ ]:
df_horario = df.melt(
    id_vars= ['FECHA'],
    value_vars= [f'DEM_GCRNO_H{i}' for i in range(24)],
    var_name="Hora",
    value_name="Demanda"
).replace(
    {f'DEM_GCRNO_H{i}': i for i in range(24)}
)

df_horario.index = df_horario.FECHA + pd.to_timedelta(df_horario.Hora, unit='h')
df_horario.sort_index(inplace=True)
df_horario.drop(columns=['FECHA', 'Hora'], inplace=True)

df_horario.plot(figsize=(15, 7))

# Análisis exploratorio de datos

1. ¿Cuales son las variables que interesan en un análisis diarios?
2. ¿Como se comporta la demanda máxima se acuerdo al mes?
3. ¿Que variación se tiene en demanda máxima y mínima por día de la semana?
4. ¿Que variaciones hay en el consumo integrado por hora?



In [ ]:
# Agrega cuantas celdas creas necesarias

## Forecasting

1. Definir un problema de *forecasting*
2. Probar con el uso de *skforcast*
3. Ajuster primero un modelo sencillo (`Ridge`) y luego ver que pasa con un modelo más complejo.


In [ ]:
# Agrega cuantas celdas consideres necesarias

## Una probadita a Prophet en rápido y furioso

In [ ]:
!pip install fbprophet

In [ ]:
from fbprophet import Prophet

df_prophet = pd.DataFrame(
    {
        'ds': df_horario.index,
        'y': df_horario.Demanda
    }
)

modelo = Prophet().fit(df_prophet[df_prophet.ds.dt.year < 2021])

In [ ]:
def stan_init(m):
    res = {}
    for pname in ['k', 'm', 'sigma_obs']:
        res[pname] = m.params[pname][0][0]
    for pname in ['delta', 'beta']:
        res[pname] = m.params[pname][0]
    return res

In [ ]:
future = modelo.make_future_dataframe(24, freq='h', include_history=False)
forecast = modelo.predict(future)

while forecast.ds.max() < df_prophet.ds.max():
  modelo = Prophet().fit(df_prophet[df_prophet.ds <= future.ds.max()], init=stan_init(modelo))
  future = modelo.make_future_dataframe(24, freq='h', include_history=False)
  forecast2 = modelo.predict(future)
  forecast = pd.concat([forecast, forecast2])
  

In [ ]:
df_val = df_prophet[df_prophet.ds.dt.year==2021]

plt.plot(forecast.ds, forecast.yhat, label='y_pred')
plt.plot(df_val.ds, df_val.y, label='y_true')
plt.legend()
plt.show()

In [ ]:
semana = 10

plt.plot(
    forecast.ds[forecast.ds.dt.weekofyear == semana], 
    forecast.yhat[forecast.ds.dt.weekofyear == semana], 
    label='y_pred')
plt.plot(
    df_val.ds[df_val.ds.dt.weekofyear == semana], 
    df_val.y[df_val.ds.dt.weekofyear == semana], 
    label='y_true')
plt.legend()
plt.show()